In [2]:
import torch

In [3]:
x = torch.tensor([ 
    [1.0,0.0,1.0,0.0],
    [0.0,2.0,0.0,2.0],
    [1.0,1.0,1.0,1.0],
])



w_query=torch.tensor([ 
    [1.0,0.0,1.0],
    [1.0,0.0,0.0],
    [0.0,0.0,1.0],
    [0.0,1.0,1.0]
])

w_key=torch.tensor([ 
    [0.0,0.0,1.0],
    [1.0,1.0,0.0],
    [0.0,1.0,0.0],
    [1.0,1.0,0.0]
])

w_value = torch.tensor([ 
    [0.0,2.0,0.0], 
    [0.0,3.0,0.0],
    [1.0,0.0,3.0],
    [1.0,1.0,0.0]
])


다음 코드는 입력 벡터 시퀀스로


쿼리,키,밸류 벡터들을 만드는 부분입니다.

torch.matlul은 행렬곱을 수행하는 함수입니다.

In [4]:
keys = torch.matmul(x,w_key)
querys = torch.matmul(x,w_query)
values = torch.matmul(x,w_value)

다음 코드는 앞 코드에서 만든 쿼리 벡터(querys)와 키 벡터(keys)를 행렬곱해서

어텐션 스코어를 만드는 과정입니다. 

keys.T는 키 벡터들을 전치한 행렬입니다.

In [5]:
attn_scores = torch.matmul(querys,keys.T)


In [6]:
attn_scores

tensor([[ 2.,  4.,  4.],
        [ 4., 16., 12.],
        [ 4., 12., 10.]])

In [9]:
import numpy as np 
from torch.nn.functional import softmax 

key_dim_sqrt = np.sqrt(keys.shape[-1])

attn_probs = softmax(attn_scores / key_dim_sqrt,dim=-1)

In [17]:
attn_probs

tensor([[1.3613e-01, 4.3194e-01, 4.3194e-01],
        [8.9045e-04, 9.0884e-01, 9.0267e-02],
        [7.4449e-03, 7.5471e-01, 2.3785e-01]])

다음 코드는 앞에서 구한 소프트맥스 확률과 밸류 벡터들을 가중합하는 과정을 수행합니다.

In [18]:
weighted_values = torch.matmul(attn_probs,values)

In [20]:

weighted_values

tensor([[1.8639, 6.3194, 1.7042],
        [1.9991, 7.8141, 0.2735],
        [1.9926, 7.4796, 0.7359]])

이 weighted_values의 shape은 (단어의 수 * 차원(value의 가로 길이))와 일치합니다.

멀티 헤드 어텐션은 이 과정을 헤드의 수 만큼 진행합니다. 

weighted_values * H 를 하면 (단어의 수 * 차원) * (헤드의 개수)가 되고, 

여기에 W0을 matmul을 하면 된다. W0.shape = (헤드의 개수) * (차원)

이 W0도 다른 Wq Wk Wv와 마찬가지로 태스크를 가장 잘 수행하는 방향으로 업데이트 됩니다.


#### Feed Forward Neural Network

In [21]:
import torch 

x = torch.tensor([2,1])
w1 = torch.tensor([ 
    [3,2,-4], 
    [2,-3,1]
])
b1 = 1 
w2 = torch.tensor([ 
    [-1,1], 
    [1,2],
    [3,1]
])
b2= -1

실제 계산을 수행하는 코드입니다.

입력 x와 w1을 행렬곱한 뒤 b1을 더한 것이 h_preact입니다. 

여기에 Relu를 취해 은닉층 h를 만듭니다.

마지막으로 h와 w2를 행렬곱한 뒤 b2를 더해 출력츨 y를 계산하게 됩니다.

In [22]:
h_preact = torch.matmul(x,w1)+b1
h = torch.nn.functional.relu(h_preact)
y = torch.matmul(h,w2)+b2

#### 레이어 정규화

레이어 정규화란 미니 배치의 인스턴스(x) 별로 평균을 빼주고, 표준편차로 나누어 정규화를 수행하는 기법입니다. 

레이어 정규화를 수행하면 학습이 안정되고 그 속도가 빨라지는 등의 효과 있다고 합니다. 



In [23]:
input = torch.tensor([[1.0,2.0,3.0],[1.0,1.0,1.0]])
#배치 크기는 2, 피처의 차원 수는 3

m = torch.nn.LayerNorm(input.shape[-1])
output = m(input)

In [24]:
output

tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward>)

In [25]:
m.weight

Parameter containing:
tensor([1., 1., 1.], requires_grad=True)

#### DropOut

In [26]:
m = torch.nn.Dropout(p=0.2)

input = torch.randn(1,10)
output = m(input)

In [27]:
input

tensor([[ 0.2738, -1.2282, -0.9771,  1.8165,  1.7545, -0.3581,  1.1253, -0.3127,
         -0.3682, -0.3343]])

In [30]:
output

tensor([[ 0.3422, -1.5353, -0.0000,  2.2706,  0.0000, -0.4476,  1.4066, -0.3909,
         -0.4603, -0.4178]])